# Fake News Detector

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords # the,for,of,in,with
from nltk.stem.porter import PorterStemmer # eating, eaten,-->eat
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#### Load Data

In [2]:
news_df = pd.read_csv('train.csv')
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


## Labels
We are only Interested in Title,Author and Label Columns
* 1 - Fake News
* 0 - Real News

In [4]:
news_df.shape

(20800, 5)

In [3]:
news_df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

<h5><span style="color:aqua;font-size:50;font-weight:italic;">We have a lot of null values in Title and Author Columns, and we can't remove that rows because dataset size is not large,so we will fill it with empty space</span></h5>


In [4]:
news_df = news_df.fillna( ' ')

In [5]:
news_df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [6]:
news_df['content'] = news_df['author']+' '+news_df['title']

In [7]:
news_df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


#### Now we will deal only with content column and label column

<h5><span style="color:Violet;font-size:50;font-weight:italic;">Define Stemming Function</span></h5>

In [7]:

ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', " " , content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content= [ps.stem(word) for word in stemmed_content if not word in stopwords.words('English')]
    stemmed_content =  " ".join(stemmed_content)
    return stemmed_content

<h5><span style="color:MediumSeaGreen;font-size:50;font-weight:italic;">Apply Stemming Function to Content Column</span></h5>

In [8]:
news_df['content']= news_df['content'].apply(stemming)

In [9]:
X = news_df['content'].values
y = news_df['label'].values

<h5><span style="color:Tomato;font-size:50;font-weight:italic;">Vectorize Data</span></h5>

In [10]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [ ]:
#print(X)

<h5><span style="color:Violet;font-size:50;font-weight:italic;">Split Data into train and test sets</span></h5>

In [11]:
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,stratify=y, random_state=42)

<h5><span style="color:Coral;font-size:50;font-weight:italic;">Fit Logistics Regresssion Model</span></h5>

In [12]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [13]:
train_y_pred = model.predict(X_train)
print("train accuracy : ",accuracy_score(train_y_pred,y_train))

train accuracy :  0.9874399038461539


In [14]:
test_y_pred = model.predict(X_test)
print("test accuracy : ",accuracy_score(test_y_pred,y_test))

test accuracy :  0.9752403846153846


In [15]:
# prediction system
input_data = X_test[10]
prediction = model.predict(input_data)
if prediction[0] == 1:
    print('Fake News')
else:
    print('Not Fake News')


Fake News


In [16]:
news_df['content'][10]

'aaron klein obama organ action partner soro link indivis disrupt trump agenda'